# Practice 2. ConvLab을 이용한 Pipelined 대화 시스템 구축

# 목차

## Step 1. MultiWOZ 데이터셋을 살펴보자

## Step 2. ConvLab을 활용해 Pipelined 대화 시스템을 구축하자

## Step 3. ConvLab에서 제공하는 모듈들로 모델들을 구성 및 진단하고, 평가하자

## Additional. End-to-end Neural Pipeline (ACL 2020) 모델을 사용해보자

# Step 1. MultiWOZ 데이터셋을 살펴보자

## Step 1.0 사전설정

아래 코드를 도와주는 module들이며, 필요시 수정할 수 있습니다.

In [1]:
import json
import os
import zipfile
import sys
from collections import Counter
from nltk.tokenize import word_tokenize

from textwrap import indent
from pprint import pformat
from pprint import pprint

CUDA_IDX = '0'
os.environ['CUDA_VISIBLE_DEVICES'] = CUDA_IDX

def read_zipped_json(filepath, filename):
    archive = zipfile.ZipFile(filepath, 'r')
    return json.load(archive.open(filename))

def pprint_manual(user_manual, name):
    """print user manual
        argument 'name' is needed to discriminate 'WOZ' from others
    """
    print('    User manual (message) : ')
    if 'WOZ' in name:
        print(" "*8, user_manual)
    else:
        for manual_one in user_manual:            print(" "*8, manual_one)


def pprint_goal(goal, name):
    """print user's goal
        argument 'name' is needed to discriminate "WOZ" from others.
    """
    if 'WOZ' in name:
        pass
    else:
        for i, mes in enumerate(goal['message']):
            mes = mes.replace('<span class=\'emphasis\'>', '')
            mes = mes.replace('</span>', '')
            goal['message'][i] = mes

    print("[Goals]")
    user_manual = None
    for key, value in goal.items():
        if not value:           # empty
            continue
        elif key == 'message':  # user manual
            user_manual = value
        else:                   # valid domain
            domain = key        
            print(indent(pformat({domain : value}), ' '*4))
    pprint_manual(user_manual, name)  

def get_valid_domains(goal):
    """return valid domains for pretty print"""
    domains = []
    for key, value in goal.items():
        if not value:           # empty
            continue
        elif key == 'message':  # user manual
            continue
        else:                   # valid domain
            domains.append(key)
    return domains

def pprint_turns(log, domains):
    """pretty print for dialogue turns"""
    
    # signal for stopping print
    signal = None
    
    for i, log_one in enumerate(log):
        
        # dummy input function for pausing
        print('-' * 20 + '1. Enter to keep going 2. Type \'stop\' and Enter to stop printing ' + '-' * 40)
        signal = input()
        if 'stop' in signal:
            break

        # check whether system turn or not
        bool_sys_turn = False
        if log_one['metadata']:
            bool_sys_turn = True

        # delete span_info
        if 'span_info' in log_one:
            del log_one['span_info']

        # delete unnecessary domains
        domain_pairs = log_one['metadata']
        del_domains = []
        for dom, _ in domain_pairs.items():
            if not dom in domains:
                del_domains.append(dom)
        for dom in del_domains:
            del domain_pairs[dom]
    
        # pretty print
        if bool_sys_turn: print("[SYS]", end=" ")
        else:             print("[USR]", end=" ")
        print("(turn {})".format(i))

        log_one['1. dialogue_state'] = log_one['metadata']
        log_one['2. dialogue_act'] = log_one['dialog_act']
        log_one['3. text'] = log_one['text']
        del log_one['metadata']
        del log_one['dialog_act']
        del log_one['text']
        print(indent(pformat(log_one, width=100), ' ' * 4))
    
    # transform signal to boolean
    if 'stop' in signal:
        signal = True
    else: 
        signal = False
    return signal

### Step 1.1 MultiWOZ 데이터셋을 불러옵니다.

MultiWOZ 데이터셋은 7개의 domain ('hotel', 'train', 'attraction', 'restaurant' 'taxi', 'policy', 'hospital') 으로 구성되어 있으며, 여행정보를 얻고자 하는 'user' 와 이를 도와주는 'system'이 나누는 대화에 대한 데이터 셋입니다.
약 10,000개의 대화 뭉치로 구성되어 있으며, train, validation, test용으로 구분되어 있습니다.

아래 코드를 실행하면, MultiWOZ 데이터 셋 내 train용 데이터 이름 100개가 출력 됩니다.

In [2]:
cur_dir = os.path.abspath(os.curdir)
print("current directory :", cur_dir)
data_dir = "ConvLab-2/data/multiwoz"
processed_data_dir = os.path.join(cur_dir, 'multiwoz_data/all_data')
if not os.path.exists(processed_data_dir):
    os.makedirs(processed_data_dir)

data_key = ['train', 'val', 'test']
data = {}
for key in data_key:
    data[key] = read_zipped_json(os.path.join(data_dir, key + '.json.zip'), key + '.json')
    print('load {}.json...! '.format(os.path.join(data_dir, key)))
    print('number of dialogues : {}'.format(len(data[key])))
print()

# print available dialogue name until 100
print(list(data['train'].keys())[:100])


current directory : /home/com10/0813_dialogue_system
load ConvLab-2/data/multiwoz/train.json...! 
number of dialogues : 8434
load ConvLab-2/data/multiwoz/val.json...! 
number of dialogues : 999
load ConvLab-2/data/multiwoz/test.json...! 
number of dialogues : 1000

['SNG01856', 'SNG0129', 'MUL2168', 'SNG01445', 'MUL2105', 'PMUL1690', 'MUL2395', 'SNG0190', 'PMUL1170', 'SNG01741', 'PMUL4899', 'MUL2261', 'SSNG0348', 'MUL0784', 'MUL0886', 'PMUL2512', 'SNG0548', 'MUL1474', 'PMUL4372', 'PMUL4047', 'PMUL0151', 'MUL0586', 'PMUL3552', 'PMUL1539', 'MUL1790', 'PMUL3021', 'SNG0699', 'SNG0228', 'PMUL3296', 'MUL1434', 'PMUL2203', 'PMUL3250', 'PMUL0510', 'MUL1124', 'PMUL3719', 'SNG0297', 'PMUL2049', 'SNG01722', 'PMUL2100', 'MUL1853', 'MUL2694', 'SNG1006', 'SNG1345', 'MUL1299', 'MUL1490', 'PMUL2749', 'MUL1628', 'PMUL2202', 'SNG01450', 'SNG0131', 'SNG0984', 'PMUL1419', 'SNG0798', 'MUL0161', 'PMUL2803', 'MUL0925', 'MUL1005', 'SNG0104', 'SNG1197', 'MUL1265', 'WOZ20259', 'MUL1223', 'PMUL2596', 'MUL2037', 

## Step 1.2 데이터가 어떻게 생겼는지 살펴봅시다.

위의 출력된 데이터 이름들 중 몇개를 파이썬 리스트 안에 삽입하여 dialogue 를 확인할 수 있습니다. (ex. names = \['SNG0943', 'MUL1801'] ))

한 dialogue는 크게

1. user의 goal, (코드 상에서 'goal'))

2. system과 user의 대화, (코드 상에서 'dialogue_turns'))

로 구분 됩니다.

***

user (\[USR]])는 정의된 goal 및 manual을 읽고, 해당 goal을 달성하기 위해 대화를 수행합니다.

system (\[SYS])은 user의 goal을 알지 못하고, 대화를 통해 (1)user가 원하는 조건을 파악하고, (2)user가 원하는 정보를 제공하며, (3)필요시 예약을 수행합니다.

***

goal 내에서,

`info`는 user 입장에서, system에게 user가 원하는 조건 및 니즈를 알려주고자(inform) 하는 내용이고,

`reqt`는 user 입장에서, system에게 uesr가 요청하고자(request) 하는 내용입니다.

***

본 데이터셋의 경우 __system model을 만드는 것__을 목표로 합니다.

***

아래 코드를 실행하면 goal 및 발화를 살펴볼 수 있으며, Enter로 넘길 수 있습니다. 

그만 보고 싶다면 stop을 입력 후 Enter 합니다.

In [3]:
# You can handle dialogue_names
dialogue_names = ['SNG0943', 'MUL1801']

for name in dialogue_names:
    
    print()
    print('-' * 125)
    print("[Dialogue name] \'{}\'".format(name))

    # access datum using name
    datum = data['train'][name]
    goal = datum['goal']
    dialogue_turns = datum['log']

    # print goal and dialogue turns
    pprint_goal(goal, name)
    valid_domains = get_valid_domains(goal)
    break_signal = pprint_turns(dialogue_turns, valid_domains)  # If you don't want to see print, please comment!
    # break_signal = pprint_turns(dialogue_turns, valid_domains)    Like this!

    if break_signal:
        break

ting ----------------------------------------
[SYS] (turn 3)
    {'1. dialogue_state': {'hotel': {'book': {'booked': [], 'day': '', 'people': '', 'stay': ''},
                                     'semi': {'area': 'not mentioned',
                                              'internet': 'not mentioned',
                                              'name': 'not mentioned',
                                              'parking': 'yes',
                                              'pricerange': 'not mentioned',
                                              'stars': '4',
                                              'type': 'not mentioned'}}},
     '2. dialogue_act': {'Hotel-Request': [['Area', '?']]},
     '3. text': "Is there a particular area of the city where you 'd like to stay ?"}
--------------------1. Enter to keep going 2. Type 'stop' and Enter to stop printing ----------------------------------------
[USR] (turn 4)
    {'1. dialogue_state': {},
     '2. dialogue_act': {'Hotel-

# Step 2. ConvLab을 활용해 Pipelined 대화 시스템을 구축하자

## Step 2.0 필요한 module들을 정의합니다

아래 코드를 도와주는 module들 이며, 필요시 수정할 수 있습니다.

In [4]:
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 11.1 MB 1.8 MB/s 
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.1.0-py3-none-any.whl size=11074433 sha256=aeb12833ceb6bf651a2d8efda70ea2c1b640d39f602f85d5691521786f5224e2
  Stored in directory: /tmp/pip-ephem-wheel-cache-i2w1m6dg/wheels/59/4f/8c/0dbaab09a776d1fa3740e9465078bfd903cc22f3985382b496
Successfully built en-core-web-sm
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [5]:
# common import: convlab2.$module.$model.$dataset
from convlab2.nlu.jointBERT.multiwoz import BERTNLU
from convlab2.nlu.milu.multiwoz import MILU
from convlab2.dst.rule.multiwoz import RuleDST
from convlab2.policy.rule.multiwoz import RulePolicy
from convlab2.nlg.template.multiwoz import TemplateNLG
from convlab2.dialog_agent import BiSession, Agent, PipelineAgent
from convlab2.evaluator.multiwoz_eval import MultiWozEvaluator
from pprint import pprint
import random
import numpy as np
import torch
import spacy

import logging 
# uncessary logging block
mpl_logger = logging.getLogger('matplotlib') 
mpl_logger.setLevel(logging.WARNING) 
cntp_logger = logging.getLogger('urllib3.connectionpool')
cntp_logger.setLevel(logging.WARNING)
ttu_logger = logging.getLogger('transformers.tokenization_utils')
ttu_logger.setLevel(logging.WARNING)
tcu_logger = logging.getLogger('transformers.configuration_utils')
tcu_logger.setLevel(logging.WARNING)
tmu_logger = logging.getLogger('transformers.modeling_utils')
tmu_logger.setLevel(logging.WARNING)
logging.getLogger().setLevel(logging.INFO)
import warnings
warnings.filterwarnings("ignore")

def set_seed(r_seed):
    random.seed(r_seed)
    np.random.seed(r_seed)
    torch.manual_seed(r_seed)


## Step 2.1 Pipelined 대화 시스템 예시를 살펴봅시다.

우선, system model을 pipelined 대화 시스템으로 구성했을 때, 대화를 어떻게 수행할 수 있는지 살펴봅시다.

pipelined 대화 모델은 크게 4가지로 구성되어 있습니다.

NLU (Natural Language Understanding (언어 인식 모듈)) : 직전의 상대방 발화를 이해 및 해석 합니다.

DST (Dialogue State Tracking (대화 상태 추적 모듈)) : 현재까지 대화의 맥락을 추적하고, 변경 사항을 업데이트 합니다.

Dialogue Policy (대화 정책 모듈) : 다음 발화를 위해 구조화된 단어 형태로 정책을 결정 합니다. (자연스러운 문장의 형태가 아닌, 의도만을 결정합니다.)

NLG (Natural Language Generation (언어 생성 모듈)) : 결정된 정책을 가지고, 사람이 이해할 수 있는 자연어를 생성합니다.

-----------------

아래는 가장 기본적인 Pipelined 대화 시스템을 구성한 예시입니다.

BERT NLU : 앞선 practice 1에서 다룬 BERT NLU

RuleDST : Rule 기반 DST module

RulePolicy : Rule 기반 Policy module

TemplateNLG : Template 기반 (정해진 템플릿 위에서 단어를 채워넣는 방식) NLG module

4가지 모듈에 대해 정의를 끝마쳤다면, PipelineAgent라는 wrapper 에 씌워 sys_agent를 선언합니다.

In [6]:
spacy.load('en_core_web_sm')
# MILU
sys_nlu = MILU()
# simple rule DST
sys_dst = RuleDST()
# rule policy
sys_policy = RulePolicy()
# template NLG
sys_nlg = TemplateNLG(is_user=False)
# assemble
sys_agent = PipelineAgent(sys_nlu, sys_dst, sys_policy, sys_nlg, name='sys')

Load from https://convlab.blob.core.windows.net/convlab-2/milu_multiwoz_all_context.tar.gz
100%|██████████| 10943993/10943993 [00:13<00:00, 787808.83B/s]


sys_agent.response("user의 발화", print_nlu=False, print_dst=False, print_pol=False) 를 하면, user의 발화에 대한 응답을 합니다.

print_nlu, print_dst, print_pol을 True로 변경하면, 해당하는 value를 print 해볼 수 있습니다.

In [7]:
sys_agent.init_session()
sys_agent.response("I want to find a moderate hotel", print_nlu=False, print_dst=False, print_pol=False)

'Will cityroomz be alright ? Of course , would a hotel be OK ? We have 3 such places .'

In [8]:
sys_agent.response("Which type of hotel is it ?")

'It is a hotel .'

In [9]:
sys_agent.response("OK , where is its address ?")

'It is located at 74 chesterton road.'

In [10]:
sys_agent.response("Thank you !")

'You are welcome . Have a good day !.'

In [11]:
sys_agent.response("Try to find me a Chinese restaurant in south area .")

'The good luck chinese food takeaway sounds like it might be what you are looking for . The reference number is 00000003 . Yes there are 3 different places that match your description .'

In [12]:
sys_agent.response("Which kind of food it provides ?")

'It is chinese food .'

In [13]:
sys_agent.response("Book a table for 5 , this Sunday .")

'All set . Your reference number is 00000003 .'

## Step 2.2 system agent와 대화할 user simulator를 구성해봅시다.

system model의 성능을 알아보기 위해서는 user 역할을 할 수 있는 user simulator가 필요합니다. 

사람이 매번 user의 역할을 하여 대화를 주고 받는 것은 많은 노동력을 필요로 하기 때문입니다. 

특히, Dialog Policy를 RL agent로 두었을 때, 다양한 대화 시도를 위해서 user simulator 는 필수적입니다.

ConvLab에서는 RulePolicy(character='usr')로 두었을 때, `Agenda` policy로 변환되며, 이는 user의 goal을 기반으로 하는 user model을 정의할 수 있습니다. 

또한, `Agenda` policy는 dst 모델까지 함께 포함하고 있기 때문에 `user_dst = None` 이 됩니다.


In [14]:
# MILU
user_nlu = MILU()
# not use dst
user_dst = None
# rule policy
user_policy = RulePolicy(character='usr')   # UserPolicyAgendaMultiWoz()
# template NLG
user_nlg = TemplateNLG(is_user=True)
# user_nlg = SCLSTM(is_user=True)
# assemble
user_agent = PipelineAgent(user_nlu, user_dst, user_policy, user_nlg, name='user')

Load from https://convlab.blob.core.windows.net/convlab-2/milu_multiwoz_all_context.tar.gz
Loading goal model is done


## Step 2.3 user simulator 와 system model 간 대화를 수행해봅시다.

지금까지, 우리는 user simulator와 system model을 정의 했습니다.

`MultiWozEvaluator` 클래스는 성능을 평가하기 위해 사용됩니다. (user의 goal을 정의해줍니다.)

`BiSession` 클래스는 user simulator와 system model의 대화 및 평가를 도와줍니다.

`next_turn` 함수는 한 턴의 대화를 수행합니다.

### 평가 지표

- Success rate : 예약 성사 + recall == 1, 즉 유저의 조건에 부합하는 예약을 해내고, 물어보는 모든 정보에 대해 알맞은 값을 출력

- Book rate : 예약 성사율 (= 예약 성공 수 / 예약 정답 수) 

- Inform precision : (TP) / (TP + FP), precision이 낮다는 것은 요청한 slot 외 불필요한 정보를 많이 알려주는 것으로 해석할 수 있음.

- Inform recall : (TP) / (TP + FN), recall이 낮다는 것은 요청한 slot에 대해 대답하지 못한 것으로 해석 할 수 있음.

- Inform F1 : Precision & Recall에 대한 조화 평균



In [15]:
evaluator = MultiWozEvaluator()
sess = BiSession(sys_agent=sys_agent, user_agent=user_agent, kb_query=None, evaluator=evaluator)

set_seed(20200804)

sys_response = ''
sess.init_session()
print('init goal:')
pprint(sess.evaluator.goal)
print('-'*100)
for i in range(20):
    sys_response, user_response, session_over, reward = sess.next_turn(sys_response)
    print('user:', user_response)
    print('sys:', sys_response)
    print()
    if session_over is True:
        break
print('task success:', sess.evaluator.task_success())
print('book rate:', sess.evaluator.book_rate())
print('inform precision/recall/f1:', sess.evaluator.inform_F1())
print('-'*50)
print('final goal:')
pprint(sess.evaluator.goal)

print('='*100)

init goal:
{'attraction': {'info': {'area': 'centre', 'type': 'museum'},
                'reqt': {'address': '?', 'phone': '?'}},
 'train': {'info': {'arriveBy': '08:30',
                    'day': 'monday',
                    'departure': 'cambridge',
                    'destination': 'peterborough'},
           'reqt': {'price': '?', 'trainID': '?'}}}
----------------------------------------------------------------------------------------------------
user: I also need a train. I need some information on a train going to peterborough . I would like to leave on monday. I 'd like to arrive by 08:30 .
sys: Where are you departing from ?

user: I ' m also looking for a train from cambridge .
sys: Would you like me to book the 08:30 train ? I can book your tickets for monday . Woudl you like me to book a train to peterborough for you ? Woudl you like me to book a train from cambridge for you ?

user: I need to book a train to peterborough .
sys: I can get you tickets for an arrival time 

# Step 3. ConvLab에서 제공하는 모듈들로 모델들을 구성 및 진단하고, 평가하자

## Step 3.0. ConvLab 에서 지원하는 모델들을 load 합니다.

이용가능한 model들:

- NLU: BERTNLU, MILU, SVMNLU
- DST: RuleDST
- Word-DST: SUMBT, TRADE (set `sys_nlu` to `None`)
- Policy: RulePolicy, Imitation, REINFORCE, PPO, GDPL
- Word-Policy: MDRG, HDSA, LaRL (set `sys_nlg` to `None`)
- NLG: Template, SCLSTM
- End2End: Sequicity, DAMD, RNN_rollout (directly used as `sys_agent`)
- Simulator policy: Agenda, VHUS (for `user_policy`)


In [16]:
# available NLU models
from convlab2.nlu.svm.multiwoz import SVMNLU
from convlab2.nlu.jointBERT.multiwoz import BERTNLU
from convlab2.nlu.milu.multiwoz import MILU
# available DST models
from convlab2.dst.rule.multiwoz import RuleDST
#from convlab2.dst.mdbt.multiwoz import MDBT
from convlab2.dst.sumbt.multiwoz import SUMBT
from convlab2.dst.trade.multiwoz import TRADE
# available Policy models
from convlab2.policy.rule.multiwoz import RulePolicy
from convlab2.policy.ppo.multiwoz import PPOPolicy
from convlab2.policy.pg.multiwoz import PGPolicy
from convlab2.policy.mle.multiwoz import MLEPolicy
from convlab2.policy.gdpl.multiwoz import GDPLPolicy
#from convlab2.policy.vhus.multiwoz import UserPolicyVHUS
from convlab2.policy.mdrg.multiwoz import MDRGWordPolicy
from convlab2.policy.hdsa.multiwoz import HDSA
from convlab2.policy.larl.multiwoz import LaRL
# available NLG models
from convlab2.nlg.template.multiwoz import TemplateNLG
from convlab2.nlg.sclstm.multiwoz import SCLSTM
# available E2E models
from convlab2.e2e.sequicity.multiwoz import Sequicity
from convlab2.e2e.damd.multiwoz import Damd

Load from https://convlab.blob.core.windows.net/convlab-2/mdrg_model.zip
08/13/2020 16:52:35 - INFO - convlab2.util.allennlp_file_utils -   https://convlab.blob.core.windows.net/convlab-2/mdrg_model.zip not found in cache, downloading to /tmp/tmpel6_7ojg
100%|██████████| 21577107/21577107 [00:14<00:00, 1454970.50B/s]
08/13/2020 16:52:50 - INFO - convlab2.util.allennlp_file_utils -   copying /tmp/tmpel6_7ojg to cache at /home/com10/.convlab2/cache/b0bc758ff68dc79ef5287ddd38b6267f8784df273b2e6f7e496a1e9031c65ca5.ea9a4a5a9034b22be1093ea89deb230956f14487e2c2441b9ee59cef0fc252a2
08/13/2020 16:52:50 - INFO - convlab2.util.allennlp_file_utils -   creating metadata file for /home/com10/.convlab2/cache/b0bc758ff68dc79ef5287ddd38b6267f8784df273b2e6f7e496a1e9031c65ca5.ea9a4a5a9034b22be1093ea89deb230956f14487e2c2441b9ee59cef0fc252a2
08/13/2020 16:52:50 - INFO - convlab2.util.allennlp_file_utils -   removing temp file /tmp/tmpel6_7ojg
Extracting...
Load from https://convlab.blob.core.windows.net/co

## Step 3.1. ConvLab에서 지원하는 모델들을 가지고 나만의 대화시스템을 만들어 봅시다.

Word-DST 모델들은 NLU와 DST가 합쳐진 모델을 의미합니다. 따라서, 별도의 NLU 모델 없이 사용할 수 있습니다.

따라서, (1) NLU+RuleDST 또는 (2) Word-DST 로 조합이 가능합니다.

**\[주의!]** Word-DST 의 경우 sys_nlu = None 이어야 합니다.

Word-Policy 모델들은 Dialogue Policy 와 NLG 가 합쳐진 모델을 의미합니다. 따라서 별도의 NLG 모델없이 사용할 수 있습니다.

따라서, (1) Policy+NLG 또는 Word-Policy 로 조합이 가능합니다.

**\[주의!]** Word-Policy 의 경우 sys_nlg = None 이어야 합니다.

`PipelineAgent` class를 이용해 Pipelined 대화 시스템을 만들 수 있습니다. 또는 End-to-End model를 사용할 수도 있습니다.



In [17]:
# NLU+RuleDST:
sys_nlu = MILU()
# sys_nlu = SVMNLU()
# sys_nlu = BERTNLU()
sys_dst = RuleDST()

# or Word-DST:
# sys_nlu = None
# sys_dst = SUMBT()
# sys_dst = TRADE()
#### (not working!) sys_dst = MDBT()

# [Caution] In Word-DST case, sys_nlu must be "None"

# Policy+NLG:
sys_policy = RulePolicy()
# sys_policy = PPOPolicy()
# sys_policy = PGPolicy()
# sys_policy = MLEPolicy()
# sys_policy = GDPLPolicy()
sys_nlg = TemplateNLG(is_user=False)
#sys_nlg = SCLSTM(is_user=False)

# or Word-Policy:
# sys_policy = LaRL()
# sys_policy = HDSA()
# sys_policy = MDRGWordPolicy()
# sys_nlg = None

# [Caution] "In Word-policy case, sys_nlg must be None"

sys_agent = PipelineAgent(sys_nlu, sys_dst, sys_policy, sys_nlg, 'sys')
# sys_agent = Sequicity()
# sys_agent = Damd()

irectional': True, 'dropout': 0.5, 'hidden_size': 200, 'input_size': 400, 'num_layers': 1, 'type': 'lstm'}, 'label_encoding': 'BIO', 'regularizer': [['scalar_parameters', {'alpha': 0.1, 'type': 'l2'}]], 'text_field_embedder': {'token_embedders': {'token_characters': {'embedding': {'embedding_dim': 16}, 'encoder': {'conv_layer_activation': 'relu', 'embedding_dim': 16, 'ngram_filter_sizes': [3], 'num_filters': 128, 'type': 'cnn'}, 'type': 'character_encoding'}, 'tokens': {'embedding_dim': 50, 'trainable': True, 'type': 'embedding'}}}, 'type': 'milu'} and extras {'vocab'}
08/13/2020 16:56:12 - INFO - allennlp.common.params -   model.type = milu
08/13/2020 16:56:12 - INFO - allennlp.common.from_params -   instantiating class <class 'convlab2.nlu.milu.model.MILU'> from params {'attention': {'matrix_dim': 400, 'type': 'bilinear', 'vector_dim': 400}, 'attention_for_intent': False, 'attention_for_tag': False, 'context_for_intent': True, 'context_for_tag': False, 'dropout': 0.3, 'encoder': {'bi

앞에서 했던 방식대로, user simulator도 정의해줍니다.

(ConvLab에서는 RulePolicy(character='usr')로 두었을 때, `Agenda` policy로 변환되며, 이는 user의 goal을 기반으로 하는 user model을 정의할 수 있습니다. )

In [18]:
user_nlu = BERTNLU()
# user_nlu = MILU()
# user_nlu = SVMNLU()
user_dst = None
user_policy = RulePolicy(character='usr')
# user_policy = UserPolicyVHUS(load_from_zip=True)
user_nlg = TemplateNLG(is_user=True)
# user_nlg = SCLSTM(is_user=True)
user_agent = PipelineAgent(user_nlu, user_dst, user_policy, user_nlg, name='user')

load train, size 8434
load val, size 999
load test, size 1000
loaded train, size 113500
loaded val, size 14730
loaded test, size 14744
dialog act num: 36
sentence label num: 137
tag num: 331
intent num: 137
tag num: 331
Load from model_file param
Load from https://convlab.blob.core.windows.net/convlab-2/bert_multiwoz_all_context.zip
08/13/2020 16:59:07 - INFO - convlab2.util.allennlp_file_utils -   https://convlab.blob.core.windows.net/convlab-2/bert_multiwoz_all_context.zip not found in cache, downloading to /tmp/tmpf9ea4aoa
100%|██████████| 425713245/425713245 [07:00<00:00, 1012736.71B/s]
08/13/2020 17:06:08 - INFO - convlab2.util.allennlp_file_utils -   copying /tmp/tmpf9ea4aoa to cache at /home/com10/.convlab2/cache/fe2b28201c498bb510ae89111e6bb1710a013c23920583adafc3f2b140376b90.4fbb3a3c9025fd8bc79740ec9ac9f931fb88de0cece837a569fba203fa3df2a0
08/13/2020 17:06:08 - INFO - convlab2.util.allennlp_file_utils -   creating metadata file for /home/com10/.convlab2/cache/fe2b28201c498bb510

## Step 3.2 분석 툴을 사용해 system model을 진단해봅시다.

Convlab2에서는 분석 툴(analysis tool)을 제공하며, 이를 통해 정의한 system model의 성능 및 취약점을 분석하고 진단할 수 있습니다.

뿐만 아니라 HTML report를 작성해주어, 조금 더 풍부한 통계 정보를 얻어낼 수 있습니다. (results/\$model_name\$ 를 참조합니다)

In [19]:
from convlab2.util.analysis_tool.analyzer import Analyzer

# if sys_nlu!=None, set use_nlu=True to collect more information
analyzer = Analyzer(user_agent=user_agent, dataset='multiwoz')

set_seed(20200131)
analyzer.comprehensive_analyze(sys_agent=sys_agent, model_name='sys_agent', total_dialog=20)

dialogue:   5%|▌         | 1/20 [00:00<00:13,  1.40it/s]08/13/2020 17:06:31 - DEBUG - root -   Value not found in standard value set: [47932720631] (slot: phone domain: taxi)
08/13/2020 17:06:31 - DEBUG - root -   Value not found in standard value set: [57253443034] (slot: phone domain: taxi)
08/13/2020 17:06:31 - DEBUG - root -   Value not found in standard value set: [31316650087] (slot: phone domain: taxi)
08/13/2020 17:06:31 - DEBUG - root -   Value not found in standard value set: [85878627496] (slot: phone domain: taxi)
08/13/2020 17:06:31 - DEBUG - root -   Value not found in standard value set: [07233561887] (slot: phone domain: taxi)
08/13/2020 17:06:31 - DEBUG - root -   Value not found in standard value set: [17343656291] (slot: phone domain: taxi)
08/13/2020 17:06:31 - DEBUG - root -   Value not found in standard value set: [65450290121] (slot: phone domain: taxi)
08/13/2020 17:06:31 - DEBUG - root -   Value not found in standard value set: [87288110405] (slot: phone domain

(0.7,
 0.65,
 0.7517857142857143,
 0.8025,
 0.7711996336996336,
 0.9761904761904762,
 18.9)

## Step 3.3 여러 개의 system model 간 성능을 비교해봅시다. 

서로 다른 3가지의 system model에 대한 결과를 아래에 채워봅시다. (vscode 기준, 더블 클릭하면 수정할 수 있습니다.)

NLU       | DST       | Policy    | NLG          | Success rate | Book rate | Inform P | Inform R | Inform F1 | Turn(succ/all) |
--------- | --------- | --------- | :----------: | :----------: | --------- | -------- | --------- | -------- | -------------- |
-         | -         | -         | -            | -            | -         | -        | -         | -        | -              |
-         | -         | -         | -            | -            | -         | -        | -         | -        | -              |
-         | -         | -         | -            | -            | -         | -        | -         | -        | -              |

In [24]:
set_seed(20200805)
# define your own system agent2
# sys_agent2 = PipelineAgent(...)
# define your own system agent3
# sys_agent3 = PipelineAgent(...)

milu = MILU()
rule_dst = RuleDST()
ppo_policy = PPOPolicy()
rule_policy = RulePolicy()
gdpl_policy = GDPLPolicy()
tnlg = TemplateNLG(is_user=False)

# sys_agent = PipelineAgent(sys_nlu, sys_dst, sys_policy, sys_nlg, 'sys')

sys_agent1 = PipelineAgent(milu, rule_dst, ppo_policy, tnlg, 'sys')
sys_agent2 = PipelineAgent(milu, rule_dst, rule_policy, tnlg, 'sys')
sys_agent3 = PipelineAgent(milu, rule_dst, gdpl_policy, tnlg, 'sys')

analyzer.compare_models(agent_list=[sys_agent, sys_agent, sys_agent], model_name=['sys_agent1', 'sys_agent2', 'sys_agent3'], total_dialog=20)

'portuguese', 'pricerange': '<nm>', 'name': '<nm>', 'area': 'north'}
system action : restaurant nooffer food portuguese area north
kb : None
current domain :  restaurant
dialogue state : {'food': 'portuguese', 'pricerange': '<nm>', 'name': '<nm>', 'area': 'south'}
system action : restaurant request restaurant inform choice two food portuguese area south
kb : {'addr': 'Cambridge Leisure Park Clifton Way', 'area': 'south', 'food': 'portuguese', 'id': '12238', 'introduction': "It's Nandos", 'location': [52.19017, 0.13699], 'name': 'nandos', 'phone': '01223327908', 'post': 'cb17dy', 'pricerange': 'cheap', 'type': 'restaurant', 'ref': '00000041'}
current domain :  restaurant
dialogue state : {'food': 'portuguese', 'pricerange': '<nm>', 'name': '<nm>', 'area': 'south'}
system action : booking inform none none restaurant recommend name nandos food portuguese area south
kb : {'addr': 'Cambridge Leisure Park Clifton Way', 'area': 'south', 'food': 'portuguese', 'id': '12238', 'introduction': "It

# Additional. End-to-end Neural Pipeline (ACL 2020) 모델을 사용해보자

Paper : Donghoon Ham *, Jeong-Gwan Lee *, Youngsoo Jang, and Kee-Eung Kim. 2020. End-to-End Neural Pipeline for Goal-Oriented Dialogue System using GPT-2. ACL 2020

![Model architecture](image/e2e_model.png)

우선, Convlab2에 있는 모델을 import 하고 multiwoz로 pretrained된 weight를 다운로드 합니다.

In [21]:
from convlab2.e2e.Transformer import Transformer
sys_agent = Transformer()

08/13/2020 17:08:46 - INFO - convlab2.e2e.Transformer.pytorch_transformers.modeling_bert -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
08/13/2020 17:08:46 - INFO - convlab2.e2e.Transformer.pytorch_transformers.modeling_xlnet -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
download model file...
Unzipping...08/13/2020 17:10:42 - INFO - convlab2.e2e.Transformer.pytorch_transformers.modeling_utils -   loading configuration file ./models/v1/config.json
08/13/2020 17:10:42 - INFO - convlab2.e2e.Transformer.pytorch_transformers.modeling_utils -   Model config {
  "attn_pdrop": 0.1,
  "embd_pdrop": 0.1,
  "finetuning_task": null,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_layer": 12,
  "n_positions": 1024,
  "num_labels": 1,
  "output_attentions": true,
  "output_hidden_states": false,
  "resid_pdrop": 0.1,
  "summary_activatio

위에서 사용했던 다른 e2e agent 와는 다르게, neural pipeline 모델은 dialogue state, system action (dialogue policy)을 확인할 수 있습니다.

In [22]:
sys_agent.init_session()
sys_agent.response("I want to find a moderate hotel")

current domain :  hotel
dialogue state : {'name': '<nm>', 'area': '<nm>', 'parking': '<nm>', 'pricerange': 'moderate', 'stars': '<nm>', 'internet': '<nm>', 'type': '<nm> '}
system action : hotel request hotel inform choice many
kb : {'addr': '124 tenison road', 'area': 'east', 'internet': 'yes', 'parking': 'no', 'id': '0', 'location': [52.1963733, 0.1987426], 'name': 'a and b guest house', 'phone': '01223315702', 'post': 'cb12dp', 'price': {'double': '70', 'family': '90', 'single': '50'}, 'pricerange': 'moderate', 'stars': '4', 'takesbookings': 'yes', 'type': 'guesthouse', 'ref': '00000000'}


'There are many options. Is there a particular area you would like to stay in?'

## Neural pipeline 모델과 대화해봅시다 !

In [23]:
sys_agent.init_session()
while True:
    raw_text = input(">>> ")
    while not raw_text:
        print('not empty')
        raw_text = input(">>> ")
    if raw_text == 'r':
        sys_agent.init_session()
        continue
    if raw_text == 'stop':
        break
    out_text = sys_agent.response(raw_text)
    print('sys: ', out_text)

current domain :  restaurant
dialogue state : {'food': '<nm>', 'pricerange': '<nm>', 'name': '<nm>', 'area': '<nm> '}
system action : restaurant request
kb : {'addr': 'Regent Street City Centre', 'area': 'centre', 'food': 'italian', 'id': '19210', 'introduction': 'Pizza hut is a large chain with restaurants nationwide offering convenience pizzas pasta and salads to eat in or take away', 'location': [52.20103, 0.126023], 'name': 'pizza hut city centre', 'phone': '01223323737', 'post': 'cb21ab', 'pricerange': 'cheap', 'type': 'restaurant', 'ref': '00000000'}
sys:  Do you have a preference on the price range?
current domain :  restaurant
dialogue state : {'food': '<nm>', 'pricerange': '<nm>', 'name': '<nm>', 'area': '<nm> '}
system action : restaurant request
kb : {'addr': 'Regent Street City Centre', 'area': 'centre', 'food': 'italian', 'id': '19210', 'introduction': 'Pizza hut is a large chain with restaurants nationwide offering convenience pizzas pasta and salads to eat in or take awa

KeyboardInterrupt: Interrupted by user